In [78]:
import requests
import pandas as pd
import ast
import json
from my_utils import get_url, get_source_pmid
import my_config

ImportError: cannot import name 'get_source_pmid' from 'my_utils' (/Users/ShokenLEE/Desktop/Codes/git/Nuclear_proteome/my_utils.py)

In [2]:
df = pd.read_csv('./Output/UniprotSubcellResults.csv')

In [89]:
df.head(n=10)

,UniprotID,Results,Uniprot_NEorER,Uniprot_subcell
0,Q92604,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,Endoplasmic reticulum membrane
1,Q9Y3T9,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Nucleus, nucleoplasm, Nucleus, nucleolus"
2,Q8NBX0,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,
3,A0A384NPM7,"{'entryType': 'UniProtKB unreviewed (TrEMBL)',...",0.0,
4,Q5VTL8,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,Nucleus
5,Q9BTX1,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Nucleus, nuclear pore complex, Nucleus membrane"
6,Q8NFQ8,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Endoplasmic reticulum membrane, Nucleus membrane"
7,Q9NXE4,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,1.0,"Endoplasmic reticulum membrane, Golgi apparatu..."
8,A0AV96,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,0.0,"Nucleus, Cytoplasm"
9,Q7LBC6,{'entryType': 'UniProtKB reviewed (Swiss-Prot)...,NaN,NaN


In [90]:
result = df.iloc[9, 1]
result = ast.literal_eval(result)

In [5]:
NE_list = ['Nucleus outer membrane','Nucleus membrane','Nucleus inner membrane','Nucleus, nuclear pore complex', 'Nucleus envelope', 'Nucleus lamina']
ER_list = ['Endoplasmic reticulum membrane','Endoplasmic reticulum','Sarcoplasmic reticulum membrane',
            'Endoplasmic reticulum-Golgi intermediate compartment membrane', 'Endoplasmic reticulum lumen']
NE_ER_list = NE_list + ER_list

good_evidence = ['ECO:0000269', # experimental
                 'ECO:0000305', # inferrence from paper
                 'ECO:0000250', # seq similarity
                 'ECO:0000255', # seq model
                 'ECO:0000312', # imported from other database
                 'ECO:0007744'] # a combination of experimental and computational evidence

five_papers = ['12958361', '20693407', '20876400', '22990521', '31142202']

In [91]:
print(json.dumps(result, indent=2))

{
  "entryType": "UniProtKB reviewed (Swiss-Prot)",
  "primaryAccession": "Q7LBC6",
  "comments": [
    {
      "commentType": "SUBCELLULAR LOCATION",
      "subcellularLocations": [
        {
          "location": {
            "evidences": [
              {
                "evidenceCode": "ECO:0000250"
              }
            ],
            "value": "Nucleus",
            "id": "SL-0191"
          }
        }
      ]
    }
  ]
}


In [81]:
# Store location names and judging if it is reliablly NE/ER
locations_list = []
judge_list = []
if 'comments' in result:
    for comment in result['comments']: # Each comment contains information for each isoform, if any
        if comment['commentType'] == 'SUBCELLULAR LOCATION':
            locs = comment.get('subcellularLocations', [])

            for loc in locs:
                if 'location' in loc:
                    # For judging this specifc location
                    location_judge = 0
                    evidence_judge = 0

                    # Store evidence code and pmid for this location
                    # Used for judging and not exported outside the loc loop
                    code_list = []
                    pmid_list = list()

                    value = loc['location'].get('value', '')
                    evidences = loc['location'].get('evidences', '')

                    # Loop over all evidences
                    for evidence in evidences:
                        code = evidence.get('evidenceCode', '')
                        pmid = evidence.get('id', '')

                        # if reliable evidence is present
                        # then the evidence is re-written to be valid
                        if code in good_evidence:
                            evidence_judge = 1

                        # if the evidence is "similarity"
                        # then the source pmid is pulled
                        if code == "ECO:0000250":
                            pmid = get_source_pmid(pmid, value)

                        code_list.append(code)
                        # Append the give PMIDs depending on type of pmid
                        # In default it is a string
                        # But if it was updated by "Similarity" then it is a list
                        if type(pmid) == str: pmid_list.append(pmid)
                        if type(pmid) == list: pmid_list += pmid

                    # if the location evidence comes only from any of 5 papers
                    # then the evidence is considered invalid
                    if len(evidences) == 1 & all(item in five_papers for item in pmid_list):
                        evidence_judge = 0
                        print("NE/ER location evidence was only from 5 papers")

                    # Final judge for this location
                    if (value in NE_ER_list) & evidence_judge == 1:
                        location_judge = 1

                # the end of loc loop, appending the location and final judge
                locations_list.append(value)
                judge_list.append(location_judge)

# Final judge
# If there is at least one "1" in judge_list
# then this protein NE/ER is suported by Uniprot but not by 5 papers
if 1 in judge_list:
    final_judge = 1
else: final_judge = 0

print(locations_list)
print(judge_list)
print(final_judge)

NE/ER location evidence was only from 5 papers
['Cytoplasm', 'Endoplasmic reticulum membrane', 'Mitochondrion membrane', 'Nucleus envelope', 'Cytoplasm', 'Nucleus']
[0, 1, 0, 0, 0, 0]
1


In [88]:
for i in range(len(df)):
    result = df.iloc[i, 1]
    result = ast.literal_eval(result)

    # Store location names and judging if it is reliablly NE/ER
    locations_list = []
    judge_list = []
    if 'comments' in result:
        for comment in result['comments']: # Each comment contains information for each isoform, if any
            if comment['commentType'] == 'SUBCELLULAR LOCATION':
                locs = comment.get('subcellularLocations', [])

                for loc in locs:
                    if 'location' in loc:
                        # For judging this specifc location
                        location_judge = 0
                        evidence_judge = 0

                        # Store evidence code and pmid for this location
                        # Used for judging and not exported outside the loc loop
                        code_list = []
                        pmid_list = list()

                        value = loc['location'].get('value', '')
                        evidences = loc['location'].get('evidences', '')

                        # Loop over all evidences
                        for evidence in evidences:
                            code = evidence.get('evidenceCode', '')
                            pmid = evidence.get('id', '')

                            # if reliable evidence is present
                            # then the evidence is re-written to be valid
                            if code in good_evidence:
                                evidence_judge = 1

                            # if the evidence is "similarity"
                            # then the source pmid is pulled
                            if code == "ECO:0000250":
                                print("similar found", i)
                                pmid = get_source_pmid(pmid, value)

                            code_list.append(code)
                            # Append the give PMIDs depending on type of pmid
                            # In default it is a string
                            # But if it was updated by "Similarity" then it is a list
                            if type(pmid) == str: pmid_list.append(pmid)
                            if type(pmid) == list: pmid_list += pmid

                        # if the location evidence comes only from any of 5 papers
                        # then the evidence is considered invalid
                        if len(evidences) == 1 & all(item in five_papers for item in pmid_list):
                            evidence_judge = 0
                            print("NE/ER location evidence was only from 5 papers")

                        # Final judge for this location
                        if (value in NE_ER_list) & evidence_judge == 1:
                            location_judge = 1

                    # the end of loc loop, appending the location and final judge
                    locations_list.append(value)
                    judge_list.append(location_judge)
    
    # Final judge
    # If there is at least one "1" in judge_list
    # then this protein NE/ER is suported by Uniprot but not by 5 papers
    if 1 in judge_list:
        final_judge = 1
    else: final_judge = 0
    
    df.loc[i, "Uniprot_NEorER"] = final_judge
    df.loc[i, "Uniprot_subcell"] = ", ".join(locations_list)

similar found 7
similar found 9
HTTP Error for URL https://rest.uniprot.org/uniprotkb/search: 400 Bad Request


SystemExit: 1

In [79]:
sum([1, 0])

1

In [74]:
get_source_pmid("Q8BN21", 'Nucleus envelope')

['31142202']

In [86]:
def get_source_pmid(entry, loc_of_interest):
    """
    Obtain PMIDs for subcellular loc information of a given Uniprot ID.
    
    Parameters:
    - entry (str): Uniprot ID of interest.
    - loc_of_interest (str): Subcellular location name for which PMID may exist.
    
    Returns:
    - pmid_list (list).
    """
    
    pmid_list = []
    
    # Query the gene name to get the Uniprot ID and the Uniprot-registered gene name
    params = {
    "query": f'accession:{entry}',
    "fields": "cc_subcellular_location",
    "format": "json"
    }

    r = get_url(my_config.WEBSITE_API, params=params)
    result = r.json()['results'][0]
        
    if 'comments' in result:
        for comment in result['comments']: # Each comment contains information for each isoform, if any
            if comment['commentType'] == 'SUBCELLULAR LOCATION':
                locs = comment.get('subcellularLocations', [])

                for loc in locs:
                    if 'location' in loc:
                        value = loc['location'].get('value', '')

                        # If this is the location of interest
                        # then dig into pmid info
                        if value == loc_of_interest:
                            evidences = loc['location'].get('evidences', '')
                            for evidence in evidences:
                                pmid = evidence.get('id', '')
                                pmid_list.append(pmid)

    return pmid_list

In [33]:
def get_source_pmid(entry, loc_of_interst):
    """
    Obtain the PMID on which the subcellular locationa of a given protein is based.
    
    Parameters:
    - entry (str): the UniProt ID of interest
    - loc (str): the name of subcellluar location for which PMID is search for (e.g. "Nuclear envelope")
    
    Returns:
    - pmids (list): the PMID(s)
    """
    print("Querying ", entry)
    
    pmids = []
    
    params = {
        "query": f'accession:{entry}',
        "fields": "cc_subcellular_location",
        "format": "json"
    }
        
    r = get_url(my_config.WEBSITE_API, params=params)
    result = r.json()['results'][0]
    
    if 'comments' in result:
        for comment in result['comments']: # Each comment contains information for each isoform, if any
            if comment['commentType'] == 'SUBCELLULAR LOCATION':
                locs = comment.get('subcellularLocations', [])

                for loc in locs:
                    if 'location' in loc:

                        value = loc['location'].get('value', '')

                        # If the value is the location of interest
                        # If true dig into evidence
                        if value == loc_of_interst:
                            evidences = loc['location'].get('evidences', '')
                            # Loop over all evidences
                            for evidence in evidences:
                                pmids.append(evidence.get('id', ''))
    
    return pmids

In [34]:
entry = 'Q8BN21'
loc = "Nucleus envelope"

In [35]:
get_source_pmid(entry, loc)

Querying  Q8BN21


['31142202']